In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import utils as u
import importlib
importlib.reload(u)
import pandas as pd

In [3]:
mpc_ast_data = pd.read_parquet(
    "../data/MPC_Database/mpcorb_extended.parquet",
    columns=["Number", "Name", "Principal_desig", "Other_desigs", "a", "e", "i", "Node", "Peri", "M", "Epoch", "Orbit_type"]
)

mpc_comet_data = pd.read_parquet("../data/MPC_Database/cometels.parquet")

It exists 4 differents columns for the sso name in the mpc database : Number, Name, Principal_desig and Other_desigs

The cross_match function look into the 4 columns and check if the ssnamenr exists in any of these columns. 

In [4]:
mpc_ast_data[["Number", "Name", "Principal_desig", "Other_desigs"]]

,Number,Name,Principal_desig,Other_desigs
0,(1),Ceres,A801 AA,"[A899 OF, 1943 XB]"
1,(2),Pallas,A802 FA,None
2,(3),Juno,A804 RA,None
3,(4),Vesta,A807 FA,None
4,(5),Astraea,A845 XA,[1969 SE]
...,...,...,...,...
1263646,None,None,2021 EH5,None
1263647,None,None,2021 FT2,None
1263648,None,None,2021 FV2,None
1263649,None,None,2021 FB3,None


In [5]:
confirmed_sso = u.load_data(["ssnamenr"])

In [6]:
confirmed_sso[confirmed_sso["ssnamenr"] == "612803"]

,ssnamenr
12051748,612803
12051749,612803
12473322,612803
12473323,612803
12473324,612803


In [7]:
mpc_in_fink, fink_not_in_mpc = u.mpc_crossmatch(mpc_ast_data, pd.Series(confirmed_sso["ssnamenr"].unique()))

In [8]:
len(mpc_in_fink) + len(fink_not_in_mpc), len(confirmed_sso["ssnamenr"].unique()), abs(len(mpc_in_fink) + len(fink_not_in_mpc) - len(confirmed_sso["ssnamenr"].unique()))

(513571, 517611, 4040)

List of solar system objet in Fink not present in the mpc database with the simple crossmatch

In [9]:
fink_not_in_mpc

951       2004RP09
973       2008UZ02
987       2012JH01
1011      2015SW00
2485      1996SC03
            ...   
517485    2016SQ07
517487    2017YX06
517488    2018BD01
517550    2008GD00
517564    2019EJ02
Length: 9115, dtype: object

Looking into the first sso name: 2004RP09

In [10]:
sso_name_test = "2004RP09"

In [11]:
mpc_ast_data[mpc_ast_data["Number"].str[1:-1] == sso_name_test]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type


In [12]:
mpc_ast_data[mpc_ast_data["Name"].str.replace(" ", "") == sso_name_test]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type


In [13]:
mpc_ast_data[mpc_ast_data["Principal_desig"].str.replace(" ", "") == sso_name_test]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type


In [14]:
def check_sso_name(x):
    try:
        return sso_name_test in [el.replace(" ", "") for el in x]
    except:
        return False

mpc_ast_data[mpc_ast_data["Other_desigs"].apply(check_sso_name)]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type


This object is not present in the mpc_database but the tag exits in the fink database. What happend if we remove the 0 before the 9 ...

In [15]:
sso_name_test = "2004RP9"

In [16]:
mpc_ast_data[mpc_ast_data["Number"].str[1:-1] == sso_name_test]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type


In [17]:
mpc_ast_data[mpc_ast_data["Name"].str.replace(" ", "") == sso_name_test]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type


In [18]:
mpc_ast_data[mpc_ast_data["Principal_desig"].str.replace(" ", "") == sso_name_test]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type
612802,(612803),None,2004 RP9,None,2.377057,0.271692,21.31709,254.25727,99.999,8.40338,2460000.5,MBA


In [19]:
mpc_ast_data[mpc_ast_data["Other_desigs"].apply(check_sso_name)]

,Number,Name,Principal_desig,Other_desigs,a,e,i,Node,Peri,M,Epoch,Orbit_type


The name exist in the Principal_desig column. So, in the fink database, we have a different name for this object without the 0.
If we look on the MPC website, the objct exists with the name '2004 RP9' by adding a space between 2004 and RP9. 

We can used AstroQuery to check if 2004 RP9 and 2004RP09 is really the same object.

In [20]:
mpc_query_results = u.queryMPC("2004RP04")

In [21]:
mpc_query_results.keys()

Index(['absolute_magnitude', 'aphelion_distance', 'arc_length',
       'argument_of_perihelion', 'ascending_node',
       'critical_list_numbered_object', 'delta_v', 'designation', 'earth_moid',
       'eccentricity', 'epoch', 'epoch_jd', 'first_observation_date_used',
       'first_opposition_used', 'inclination', 'jupiter_moid', 'km_neo',
       'last_observation_date_used', 'last_opposition_used', 'mars_moid',
       'mean_anomaly', 'mean_daily_motion', 'mercury_moid', 'name', 'neo',
       'number', 'observations', 'oppositions', 'orbit_type',
       'orbit_uncertainty', 'p_vector_x', 'p_vector_y', 'p_vector_z',
       'perihelion_date', 'perihelion_date_jd', 'perihelion_distance',
       'period', 'pha', 'phase_slope', 'q_vector_x', 'q_vector_y',
       'q_vector_z', 'residual_rms', 'saturn_moid', 'semimajor_axis',
       'tisserand_jupiter', 'updated_at', 'uranus_moid', 'venus_moid'],
      dtype='object')

In [22]:
mpc_query_results["name"], mpc_query_results["number"]

(None, 2004)

It seems to not be the right object, I need help !!!!

## Comet Case

In [23]:
a = 0
comet_list_in_fink = []
for i in fink_not_in_mpc:
    if len(i) != 8:
        print(i)
        comet_list_in_fink.append(i)
        a += 1

print(a)

124P
2P
C/2018DO4
137P
223P
384P
10P
289P
249P
377P
395P
49P
117P
191P
376P
143P
36P
312P
P/2019LD2
2013PA104
110P
172P
311P
29P
162P
99P
409P
28P
113P
179P
324P
408P
378P
399P
230P
244P
274P
410P
100P
337P
448P
446P
286P
94P
C/2022QE78
276P
442P
121P
P/2020WJ5
49


Almost all the name seen in the above cell seems to be Comets

In [24]:
mpc_comet_data = pd.read_parquet("../data/MPC_Database/cometels.parquet")

In [25]:
mpc_comet_data[mpc_comet_data["Designation_and_name"].str.split(" ").apply(lambda x: x[0]).isin(comet_list_in_fink)]

,Orbit_type,Provisional_packed_desig,Year_of_perihelion,Month_of_perihelion,Day_of_perihelion,Perihelion_dist,e,Peri,Node,i,Epoch_year,Epoch_month,Epoch_day,H,G,Designation_and_name,Ref,Comet_num


In [26]:
test_comet = mpc_comet_data["Designation_and_name"].str.split(" ").apply(lambda x: x[0]).str.split("/").apply(lambda x: x[0]).isin(comet_list_in_fink)

In [30]:
mpc_comet_data["Designation_and_name"].str.split(" ").apply(lambda x: x[0]).str.split("/")

0                    [C, 1995]
1                    [P, 1996]
2                    [P, 1998]
3                    [P, 1999]
4                    [P, 1999]
                ...           
970             [450P, LONEOS]
971        [451P, Christensen]
972    [452P, Sheppard-Jewitt]
973            [1I, `Oumuamua]
974              [2I, Borisov]
Name: Designation_and_name, Length: 975, dtype: object

In [27]:
mpc_comet_data[test_comet]

,Orbit_type,Provisional_packed_desig,Year_of_perihelion,Month_of_perihelion,Day_of_perihelion,Perihelion_dist,e,Peri,Node,i,Epoch_year,Epoch_month,Epoch_day,H,G,Designation_and_name,Ref,Comet_num
514,P,None,2023,10,22.4327,0.339084,0.847230,187.2200,334.0961,11.3606,2023.0,1.0,17.0,11.5,6.0,2P/Encke,MPEC 2022,2.0
520,P,None,2021,3,24.1120,1.413932,0.538164,195.5233,117.8145,12.0362,2023.0,1.0,17.0,5.0,10.0,10P/Tempel,MPEC 2023,10.0
535,P,None,2021,3,11.8149,1.580299,0.773630,347.4613,346.3389,14.3112,2023.0,1.0,17.0,8.5,6.0,28P/Neujmin,MPEC 2023,28.0
536,P,None,2019,4,20.4627,5.776860,0.044738,50.9630,312.3958,9.3639,2023.0,1.0,17.0,4.0,4.0,29P/Schwassmann-Wachmann,MPEC 2023,29.0
542,P,None,2020,6,1.9037,3.023778,0.269093,201.0338,181.8408,9.9529,2023.0,1.0,17.0,8.5,6.0,36P/Whipple,MPEC 2022,36.0
555,P,None,2025,4,10.8836,1.432914,0.598561,332.9198,118.7918,19.0519,2023.0,1.0,17.0,13.4,4.0,49P/Arend-Rigaux,MPEC 2021,49.0
605,P,None,2023,5,21.0952,2.227053,0.365373,92.4219,70.8567,6.1877,2023.0,1.0,17.0,9.0,6.0,94P/Russell,MPEC 2023,94.0
610,P,None,2022,4,11.8170,4.705793,0.228798,174.8464,28.1017,4.3399,2023.0,1.0,17.0,4.5,6.0,99P/Kowal,MPEC 2022,99.0
611,P,None,2022,8,10.9288,2.017605,0.411990,181.9630,37.6880,25.5669,2023.0,1.0,17.0,9.0,8.0,100P/Hartley,MPEC 2022,100.0
621,P,None,2021,10,18.3719,2.455370,0.318669,167.4606,287.5203,11.7046,2023.0,1.0,17.0,1.0,12.0,110P/Hartley,MPEC 2022,110.0


In [29]:
len(mpc_comet_data[test_comet])

44

In [28]:
u.queryMPC("0000244", kind="comet")

aphelion_distance                                                           5.873
argument_of_perihelion                                                   93.13414
ascending_node                                                        353.9950957
computer_name                                                             Rudenko
created_at                                                   2023-01-14T08:03:15Z
designation                                                                  244P
earth_moid                                                                   None
eccentricity                                                             0.199401
epoch                                                                2023-02-25.0
epoch_jd                                                                2460000.5
first_observation_date_used                                          2000-11-29.0
fragment                                                                     None
inclination     